# Choice of $\lambda$-values
The number and distribution of $\lambda$-values influences the error in the calculation of atomic energies. This notebook investigates how the much the ML-error changes if 
- the number of $\lambda$-values is changed (adding/removing densities at certain $\lambda$-values)

The code for the generation of the data can be found in:

alchemy_tools.test_impact_lambda: calculates the atomic energies where always one $\lambda$-value (of 0.2, 0.4, 0.6, 0.8) is neglected. The data is stored in /home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/compound/no_*ve*.txt (where the neglected $\lambda = \frac{ve}{38}$

crossvalidate.choose_different_lambdas: generates crossvalidated learning curves using the atomic energies created by test_impact_lambda; the hyperparameters are the same as for the data where all $\lambda$-values are used


## ML-Error if one $\lambda$-value is left out
The plot shows learning curves if one $\lambda$-value is left out and the learning curve where all $\lambda$-values are used. ~The error is for all curves in the same range which shows that an unsufficient number of $\lambda$-values is not the main contributor to the error. (In principle using all $\lambda$-values should give the lowest error, this is not for every training set size the case. The reason for this behaviour is probably, that the number of crossvalidation samples (=10) is not large enough).~

In [ ]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 22})
import numpy as np

In [ ]:
# load learning curves

base_path = '/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/'
l_curves_data = ['learning_curves.tab', 'no_8.tab', 'no_15.tab', 'no_23.tab', 'no_30.tab']

l_curves = np.empty((5,10,3))

for idx, lc in enumerate(l_curves_data):
    l_curves[idx] = np.loadtxt(base_path+lc)


In [ ]:
# plot results
labels = ['all', 'no $\lambda_{0.2}$', 'no $\lambda_{0.4}$', 'no $\lambda_{0.6}$', 'no $\lambda_{0.8}$']

fig, ax = plt.subplots(1,1)

for idx, l in enumerate(labels):
    ax.plot(l_curves[idx][:,0], l_curves[idx][:,1], '-o', label = l)

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlabel('Training set size')
ax.set_ylabel('Mean error  (Ha)')

ax.legend()

## Change in atomisation energy if one $\lambda$-value is left out
The difference between the integrals where all $\lambda$-values are used and where $\lambda \approx 0.8$ is left out is $\approx 0.05$ Ha. This value is in the order of the minimum error that we obtain for our learning curves (0.02 Ha). This suggest, that the number of $\lambda$-values is insufficient.

In [ ]:
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies')
import qml_interface as qi
import numpy as np

import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 22})

In [ ]:
p_all = '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/finished_abs'
paths_all = qi.wrapper_alch_data(p_all)

p_no30 = '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/paths_no_30'
paths_no30 = qi.wrapper_alch_data(p_no30)


In [ ]:
# verify that same order
a=[el.rstrip('no_30.txt') for el in paths_no30]
b=[el.rstrip('atomic_energies.txt') for el in paths_all]

for idx in range(len(a)):
    assert a[idx]==b[idx]

In [ ]:
data_all, msize_all = qi.load_alchemy_data(paths_all)
data_no_30, msize_no30 = qi.load_alchemy_data(paths_no30)

In [ ]:
diff = []
for idx in range(len(data_all)):
    diff.extend(data_all[idx][:,5] - data_no_30[idx][:,5])
diff = np.array(diff)
mean_diff = np.abs(diff).mean()

In [ ]:
fig, ax = plt.subplots(1,2)
mean=[]
std = []
idx_per_charge = qi.partition_idx_by_charge(data_all, range(len(data_all)))
for i in range(len(idx_per_charge)):
    ax[0].scatter(range(len(idx_per_charge[i][1][0])),diff[idx_per_charge[i][1]], label = 'Z = {}'.format(idx_per_charge[i][0]))
#     ax[1].bar(i, np.abs(diff[idx_per_charge[i][1]]).mean()  )
    mean.append(np.abs(diff[idx_per_charge[i][1]]).mean())
    std.append(np.abs(diff[idx_per_charge[i][1]]).std())
    
ax[0].set_xlabel('atom ID')
ax[0].set_ylabel(r'$\Delta E_{atomic}$ (Ha)')
ax[0].legend()

mean.append(np.abs(diff).mean())
std.append(np.abs(diff).std())

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
ax[1].bar(range(len(idx_per_charge)+1),mean, yerr=std, tick_label=['Z = 1', 'Z = 6', 'Z = 7', 'Z = 8', 'all'], color = colors[0:len(idx_per_charge)+1])
ax[1].set_ylabel(r'$\overline{\Delta E}_{atomic}$ (Ha)')

## Convergence of the integral $\int d\lambda$
We calculate the integral for different amounts of $\lambda$-values and plot how the integral changes if more values are added.

In [ ]:
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies')

import alchemy_tools as at
import glob


In [ ]:
def get_paths(directory):
    # load data from cube files
    paths_cubes = ['/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/ueg/ve_00.cube']
    paths2 = glob.glob(directory+'/cube-files/*')
    paths2.sort()
    paths_cubes.extend(paths2)
    return(paths_cubes)
    

### $\Delta E_{atomic} = E(\text{the original 6}) - E(\text{all} \lambda)$

In [ ]:
directories = ['/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_003712',
 '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_003886',
 '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212']

In [ ]:
diff_en = []
diff_alch = []

for d in directories:
    paths_cubes = get_paths(d)
    # all lambda values
    lam_vals, densities, nuclei, gpts = at.load_cube_data(paths_cubes)
    av_dens = at.integrate_lambda_density(densities, lam_vals, method='trapz')
    atomic_energies, alch_pots = at.calculate_atomic_energies(av_dens, nuclei, gpts)
    
    # the orginal 6
    old_densities = []
    old_lam_vals = []

    for i in range(len(lam_vals)):
        if lam_vals[i] in [0, 8/38, 15/38, 23/38, 30/38, 38/38]:
            old_lam_vals.append(lam_vals[i])
            old_densities.append(densities[i])
            
    av_dens_old = at.integrate_lambda_density(old_densities, old_lam_vals, method='trapz')
    atomic_energies_old, alch_pots_old = at.calculate_atomic_energies(av_dens_old, nuclei, gpts)
    
    diff_en.append(atomic_energies_old-atomic_energies)
    diff_alch.append(alch_pots_old - alch_pots)

In [ ]:
diff_en

In [ ]:
for i in range(3):
    print(np.abs(diff_en[i]).mean())

In [ ]:
diff_alch

In [ ]:
for i in range(3):
    print(np.abs(diff_alch[i]).mean())

## $ \frac{d}{d \lambda} \rho$

In [ ]:
def get_derivatives(lam_vals, densities):
    derivatives = []
    for idx in range(1, len(densities)):
        d_dens_lam = (densities[idx] - densities[idx-1])/(lam_vals[idx]-lam_vals[idx-1])
        derivatives.append(d_dens_lam)
    
    derivatives = np.array(derivatives)
    return(derivatives)

def sum_gradients(derivatives):
    gradients = []
    for idx in range(len(derivatives)):
        sum_gradients = np.abs(derivatives[idx]).sum()
        gradients.append(sum_gradients)
    return(np.array(gradients))

In [ ]:
derivatives = get_derivatives(lam_vals, densities)

In [ ]:
gradients_sum = sum_gradients(derivatives)

In [ ]:
gradients_sum

In [ ]:
old_derivatives = get_derivatives(old_lam_vals, old_densities)
old_gradients_sum = sum_gradients(old_derivatives)

In [ ]:
old_gradients_sum

# Estimate importance of $\lambda$-value for integral
~The atomic energies are calculated by integrated over a finite amount of $\lambda$-values.
The integration error for an interval depends on the maximum gradient of the function in the interval.
Thus, the maximum gradient is helpful to identify the minimum set of $\lambda$-values, that yields atomic energies with an error below a certain threshold.
We calculate the average gradient $\bar{g}_{i, i-1}(\vec{r}_j)$ between two $\lambda$-points $\lambda_i, \lambda_{i-1}$ for all pairs of adjacent points at every gridpoint $\vec{r}_j$ as~

~\begin{equation}
\bar{g}_{i, i-1}(\vec{r}_j) = \frac{\rho(\lambda_i, \vec{r}_j) - \rho(\lambda_{i-1}, \vec{r}_j) }{ \lambda_{i} - \lambda_{i-1} },
\end{equation}
where $\rho(\lambda_i, \vec{r}_j)$ is the density for $\lambda_i$ at point $\vec{r}_j$ in space.~

~Therefore, we obtain a set of $M$ gradients $\{\bar{g}_{i, i-1}(\vec{r}_j)\}$, where $M$ is the number of gridpoints for $N-1$ pairs of adjacent $\lambda$-values, where $N$ is the number of $\lambda$-values.
An estimate for the integration error $\Delta I$ in case of trapezoidal integration is:
\begin{equation}
    \Delta I_{i, i-1}(\vec{r}_j) \propto f''_{i-1/2}(\vec{r}_j)(\lambda_{i} - \lambda_{i-1})^3,
\end{equation}
where $f''_{i-1/2}$ is the second derivative $\frac{d^2f}{d \lambda^2}$ at the midpoint between $\lambda_{i}$ and $ \lambda_{i-1} $. $f''_{i-1/2}$ can be approximated by
\begin{equation}
f''_{i-1/2} \approx \frac{ f''_{i} + f''_{i-1}}{2}.
\end{equation}
$ f''_{i} $ canbe calculated with the centratl difference scheme as
\begin{equation}
    f''_{i} = \frac{ f''_{i+1} - 2\cdot f''_{i} + f''_{i-1} }{}
\end{equation}
Then, we can estimate the error in alchemical potential $\Delta \mu$ at gridpoint $\vec{r}_j$ as
\begin{equation}
    \Delta \mu_{i, i-1, k} = \int d\vec{r} \frac{\Delta I_{i, i-1}(\vec{r})}{|\vec{r}-\vec{R}_k|},
\end{equation}
where $\vec{R}_k$ is the position of nucleus $k$ and the error in atomic energies $ \Delta \Delta E_{i, i-1} $ is
\begin{equation}
    \Delta \Delta E_{i, i-1} = \sum_k Z_k \Delta \mu_{i, i-1, k},
\end{equation}
where the $Z_k$ are the nuclear charges of the nuclei.~

We calculate $\Delta E'$ as 
\begin{equation}
    \Delta E' = \sum_I Z_I \int d\vec{r} \int_0^{\lambda'} d\lambda \frac{\rho(\lambda, \vec{r})}{|\vec{r}-\vec{R}_I|}
\end{equation}

and plot $\Delta E'$ vs $\lambda'$. These plots are constructed for two sets of $\lambda$-values with different amounts of $\lambda$-points. The difference between the curves $\Delta E'$ for the sets could help to identify the intervals in which a high number of $\lambda$-values is needed.

In [ ]:
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies')

import alchemy_tools as at
import glob
import numpy as np
import scipy

import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 24})

def get_paths(directory):
    # load data from cube files
    paths_cubes = ['/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/ueg/ve_00.cube']
    paths2 = glob.glob(directory+'/cube-files/*')
    paths2.sort()
    paths_cubes.extend(paths2)
    return(paths_cubes)

In [ ]:
# load the cube-files

# for set choose adjacent densities and lam_vals and call integrate_lambda_density, calculate_atomic_energies


In [ ]:
# load the cube-files
compound = 'dsgdb9nsd_001212'
paths_cubes = get_paths('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212')
lam_vals, densities, nuclei, gpts = at.load_cube_data(paths_cubes)

# Small grid

Integrate over six roughly evenly spaced points from 0 to 1 using the trapezoidal rule. I calculate the integral piecewise for every pair of adjacent points $a, b$.
\begin{equation}
    I_{ab, coarse} = \int_a^b d \lambda \rho(\lambda)
\end{equation}


In [ ]:
# indices of the six points on the coarse grid
idx_set_small = []
for idx, val in enumerate(lam_vals):
    if val in [0, 8/38, 15/38, 23/38, 30/38, 1]:
        idx_set_small.append(idx)

# trapezoidal

In [ ]:
# small set
# energies
e_prime = []
lambda_prime = []

for idx in range(0, 5):
    a = idx_set_small[idx]
    b = idx_set_small[idx+1]
    lambda_prime.append(lam_vals[b])
    av_dens = at.integrate_lambda_density([densities[a], densities[b]], [lam_vals[a], lam_vals[b]])
    atomic_energies, alch_pots = at.calculate_atomic_energies(av_dens, nuclei, gpts)
    e_prime.append(atomic_energies.sum())

e_prime_small = np.array(e_prime)
e_prime_cum_small = np.array(e_prime).cumsum()
lambda_prime_small_cum = np.array(lambda_prime)

In [ ]:
e_prime_small.sum()

# csplines

In [ ]:
# get lambda-vals and densities for small set of lambda points
small_densities = []
small_lam_vals = []

for idx in idx_set_small:
    small_densities.append(densities[idx])
    small_lam_vals.append(lam_vals[idx])

In [ ]:
# calculate csplines for small dataset
shape = densities[0].shape
reshaped_densities = at.reshape_densities(small_densities)
poly_obj_small = scipy.interpolate.CubicSpline(small_lam_vals, reshaped_densities, axis=0, bc_type=('clamped', 'not-a-knot'))


In [ ]:
integration_bounds = np.array([0, 8, 15, 23, 30, 38])/38
e_prime = []
for idx in range(len(integration_bounds)-1):
    averaged_density = poly_obj_small.integrate(integration_bounds[idx], integration_bounds[idx+1])
    averaged_density = np.reshape(averaged_density, shape)
    atomic_energies, alch_pots = at.calculate_atomic_energies(averaged_density, nuclei, gpts)
    e_prime.append(atomic_energies.sum())
    
e_prime_small_csplines = np.array(e_prime)
e_prime_cum_small_csplines = np.array(e_prime).cumsum()
lambda_prime_small_cum_csplines = integration_bounds[1:]

In [ ]:
e_prime_small_csplines

In [ ]:
e_prime_cum_small_csplines

# Large grid
Integrate over all points of the fine grid. Calculate the integrals piecewise between pairs of adjacent points
\begin{equation}
I_{a'b'} = \int_{a'}^{b'} d \lambda \rho^*(\lambda)
\end{equation}




# trapezoidal

In [ ]:
# large set
# energies
e_prime = []
# even_idx = np.where(np.array(lam_vals)*38%2 == 0)[0]
for idx in range(0, len(densities)-1):
    a = idx
    b = idx+1
    av_dens = at.integrate_lambda_density([densities[a], densities[b]], [lam_vals[a], lam_vals[b]])
    atomic_energies, alch_pots = at.calculate_atomic_energies(av_dens, nuclei, gpts)
    e_prime.append(atomic_energies.sum())
                        
e_prime_full = np.array(e_prime) # stepwise energies
e_prime_cum_full = np.array(e_prime).cumsum() # cumulated energies

In [ ]:
e_prime_full.sum()

In [ ]:
e_prime_cum_full

In [ ]:
# contains all lam vals except for lambda = 0
lambda_prime = []
for idx in range(0, len(lam_vals)-1):
    a = idx
    b = idx+1
    lambda_prime.append(lam_vals[b])
lambda_prime_full_cum = np.array(lambda_prime)
lambda_prime_full_cum

# csplines

In [ ]:
# calculate csplines for large dataset
shape = densities[0].shape
reshaped_densities = at.reshape_densities(densities)
poly_obj_full = scipy.interpolate.CubicSpline(lam_vals, reshaped_densities, axis=0, bc_type=('clamped', 'not-a-knot'))

In [ ]:
integration_bounds = np.array(lam_vals)
e_prime = []
for idx in range(len(integration_bounds)-1):
    averaged_density = poly_obj_full.integrate(integration_bounds[idx], integration_bounds[idx+1])
    averaged_density = np.reshape(averaged_density, shape)
    atomic_energies, alch_pots = at.calculate_atomic_energies(averaged_density, nuclei, gpts)
    e_prime.append(atomic_energies.sum())
    
e_prime_csplines = np.array(e_prime)
e_prime_cum_csplines = np.array(e_prime).cumsum()
lambda_prime_cum_csplines = integration_bounds[1:]

In [ ]:
e_prime_cum_csplines

In [ ]:
# rename cspline stuff
e_prime_small = e_prime_small_csplines
e_prime_cum_small = e_prime_cum_small_csplines
lambda_prime_small_cum = lambda_prime_small_cum_csplines

e_prime_full = e_prime_csplines # stepwise energies
e_prime_cum_full = np.array(e_prime_csplines).cumsum() # cumulated energies
lambda_prime_full_cum = integration_bounds[1:]

In [ ]:
e_prime_cum_full

## Difference fine and coarse grid
Compare the integrals over the fine grid and the coarse grid by comparing the integral values over the intervals $[a,b]$ of the coarse grid. For this comparison the integrals $I_{a'b'}$ over the fine grid must be summed up as 
\begin{equation}
I_{ab, fine} = \sum_{a,b'}^{a', b} I_{a', b'}
\end{equation}

In [ ]:
# integrals of fine grid over same lambda values as for coarse grid
delta_fine = []
for idx in range(0, len(idx_set_small)-1):
    a = lam_vals[idx_set_small[idx]] # point lower interval limit a on the coarse grid
    b = lam_vals[idx_set_small[idx+1]] # upper interval limit on the coarse grid
    
    # lambda_prime_full_cum is the lambda value up to which is integrated from 0 if the cumulative 
    # energy is given (it contains all lambda values of the fine grid except for lambda = 0)
    E_coarse = e_prime_full[np.where((lambda_prime_full_cum>a) & (lambda_prime_full_cum<=b) )].sum()
    print(np.where((lambda_prime_full_cum>a) & (lambda_prime_full_cum<=b) ))
    delta_fine.append(E_coarse)

I_fine = np.array(delta_fine)
I_fine.sum()

The difference between the integrals over the interval $[a, b]$ is then
\begin{equation}
    \Delta I = I_{ab, fine} - I_{ab, coarse}
\end{equation}

In [ ]:
delta_I = I_fine - e_prime_small
delta_I

## Plot - Difference over every interval $[a, b]$
Bar plot of integrals over the intervals $[a, b]$ on the fine and the coarse grid and the difference between  those integrals
- Shows difference of integrals over certain intervals
- the intervals where the difference is largest need biggest amount of additional data points

In [ ]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 20})

fig, ax = plt.subplots(1,2)
fig.suptitle(compound)

tick_label = ['[0.0, 0.2]', '[0.2, 0.4]', '[0.4, 0.6]', '[0.6, 0.8]', '[0.8, 1.0]']
ax[0].bar(np.arange(len(I_fine))+0.125, I_fine, width = 0.25, color='blue', alpha=0.3, tick_label = tick_label, label='fine grid')
ax[0].bar(np.arange(len(I_fine))-0.125, e_prime_small, width = 0.25, color='red', alpha=0.3, tick_label = tick_label, label='coarse grid')
ax[1].bar(np.arange(len(I_fine)), delta_I, width = 0.25, color='orange', alpha=0.3, tick_label = tick_label)

# axis labels
ax[0].set_xlabel('Integration interval')
ax[0].set_ylabel(r'$E_{ab} =  \int_a^b d \lambda \rho^*(\lambda)$ [Ha]')
ax[0].legend()

ax[1].set_xlabel('Integration interval')
ax[1].set_ylabel(r'$E_{ab, fine} -E_{ab, coarse}$ [Ha]')



## Plot - Cumulated integrals
The region where the curves has the stepest slope should be the region where the error is largest.

In [ ]:
fig, ax = plt.subplots(1,1)
ax.set_title(compound)

ax.plot(lambda_prime_full_cum, e_prime_cum_full, '-o', label='fine grid')
ax.plot(lambda_prime_small_cum, e_prime_cum_small, '-o', label='coarse grid')

# axis labels
ax.set_xlabel(r'$\lambda$')
ax.set_ylabel(r'$\Delta E = \sum_I Z_I \int_0^{\lambda} d\lambda \rho_I*(\lambda) $')
ax.legend()

## Plot - Increase over evenly spaced intervals
For the fine grid the increase of the integral over $[a,b]$ is almost linear, while it is a lot more unsteady for the coarse grid.

In [ ]:
e_prime_full

In [ ]:
e_prime_cum_small_csplines

In [ ]:
# lambda values for cumulated integral
lambda_prime = []
even_idx = np.where(np.array(lam_vals)*38%2 == 0)[0]
for idx in range(0, len(even_idx)):
    a = even_idx[idx]
    lambda_prime.append(lam_vals[a])
lambda_even_large = np.array(lambda_prime)

In [ ]:
# integrals on fine grid over evenly spaced intervals
I_even = []
for idx in range(0, len(lambda_even_large)-1):
    a = lambda_even_large[idx] 
    b = lambda_even_large[idx+1] 
    
    E_coarse = e_prime_full[np.where((lambda_prime_full_cum>a) & (lambda_prime_full_cum<=b) )].sum()

    I_even.append(E_coarse)

I_even = np.array(I_even)
I_even.sum()

In [ ]:
# value of integrals and difference between coarse and fine grid

fig, ax = plt.subplots(1,1)
ax.set_title(compound)
ax.plot(lambda_even_large[1:], I_even, '-o', label = 'fine grid')
ax.plot(lambda_prime_small_cum, e_prime_small, '-o', label = 'coarse grid')

ax.legend()
ax.set_xlabel(r'$b$')
ax.set_ylabel(r'$\Delta E = \sum_I Z_I \int_a^{b} d\lambda \rho_I*(\lambda) $')


## Example plot for projected densities at different $\lambda$-values

In [ ]:
import glob
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies')
from parse_cube_files import CUBE
import numpy as np

import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 22})

In [ ]:
f,a = plt.subplots(1,1)
a.plot(np.arange(40))

In [ ]:
p = '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212/cube-files'
paths = glob.glob(p+'/*.cube')
paths.sort()

cubes = []
for path in paths:
    cubes.append(CUBE(path))

In [ ]:
float(paths[0].split('/')[-1].split('.')[0].split('_')[1])